In [5]:
import datasets
ds = datasets.load_dataset('deepmind/code_contests', split='train')

Resolving data files:   0%|          | 0/39 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/24 [00:00<?, ?it/s]

In [7]:
def map_code_contests(example):
    example['question'] = example['description']# + '\nExample input/output pair:\nInput:' + str(example['public_tests']['input']) + '\nOutput:' + str(example['public_tests']['output'])
    #first_python = example['solutions']['language'].index(3)
    #example['solution'] = example['solutions']['solution'][first_python]
    example['test_inputs'] = [x.strip().split('\n') for x in example['private_tests']['input']]
    example['test_outputs'] = [x.strip() for x in example['private_tests']['output']]
    example['max_time'] = float(example['time_limit']['seconds'])
    return {'question': example['question'], 'test_inputs': example['test_inputs'], 'test_outputs': example['test_outputs'], 'max_time': example['time_limit']}
ds = ds.select(range(100))
ds = ds.filter(lambda ex: ex['difficulty']  == 7 and '<image>' not in ex['description']) # filter easy samples 
ds = ds.map(map_code_contests, remove_columns=ds.column_names)
        

Filter:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [9]:
list(ds[0].keys())

['question', 'test_inputs', 'test_outputs', 'max_time']

In [79]:
import io
import sys

def test_solution(test_case):
    test_input = test_case['test_inputs']
    test_output = test_case['test_outputs']
    # Define the solution code as a string
    solution = test_case['solution']
    res = []
    for i in range(len(test_input)):
        # Redirect stdout to capture print statements in the solution code
        output_capture = io.StringIO()
        original_stdout = sys.stdout
        sys.stdout = output_capture
        inp = iter(test_input[i])
        # Execute the solution code with a custom input function
        try:
            # Set up a local environment with a mock input function
            local_vars = {"input": lambda: next(inp)}

            # Execute the solution in this environment
            exec(solution, {}, local_vars)
        finally:
            # Restore the original stdout
            sys.stdout = original_stdout

        # Retrieve the captured output
        captured_output = output_capture.getvalue().strip()

        # Compare the captured output to the expected output
        if captured_output == test_output[i]:
            res.append(1)
        else:
            #print(test_case['solution'], end='------------------\n')

            res.append(0)
    return sum(res) / len(res)

for i,c in enumerate(modified_dataset):
    print(f"case number {i} has {test_solution(c)*100} % correct tests")

case number 0 has 100.0 % correct tests
case number 1 has 100.0 % correct tests
case number 2 has 63.63636363636363 % correct tests
case number 3 has 100.0 % correct tests
case number 4 has 100.0 % correct tests
case number 5 has 100.0 % correct tests
case number 6 has 100.0 % correct tests
case number 7 has 100.0 % correct tests
case number 8 has 0.0 % correct tests
case number 9 has 100.0 % correct tests
case number 10 has 100.0 % correct tests
case number 11 has 100.0 % correct tests


KeyboardInterrupt: Interrupted by user

In [81]:
print('\n'.join([f"{k}: {v}" for k,v in modified_dataset[0].items()]))

question: Let's call a string a phone number if it has length 11 and fits the pattern "8xxxxxxxxxx", where each "x" is replaced by a digit.

For example, "80123456789" and "80000000000" are phone numbers, while "8012345678" and "79000000000" are not.

You have n cards with digits, and you want to use them to make as many phone numbers as possible. Each card must be used in at most one phone number, and you don't have to use all cards. The phone numbers do not necessarily have to be distinct.

Input

The first line contains an integer n — the number of cards with digits that you have (1 ≤ n ≤ 100).

The second line contains a string of n digits (characters "0", "1", ..., "9") s_1, s_2, …, s_n. The string will not contain any other characters, such as leading or trailing spaces.

Output

If at least one phone number can be made from these cards, output the maximum number of phone numbers that can be made. Otherwise, output 0.

Examples

Input

11
00000000008


Output

1


Input

22
00112